<a href="https://colab.research.google.com/github/wysdi/colab-notebook/blob/main/Shopee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext google.colab.data_table

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium



Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Hit:14 http://ppa.launchpad.

In [ ]:
!pip install gspread


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

from random import randint
import pandas as pd
import numpy as np
import time
import re
import gspread
from gspread.exceptions import WorksheetNotFound


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# open it, go to a website, and get results
driver= webdriver.Chrome('chromedriver',options=options)
driver.set_page_load_timeout(25)




WebDriverException: ignored

In [ ]:
class Shopee_scraper:

    def seller_products(self, seller_name):
        seller_url = '{}{}'.format('https://shopee.co.id/', seller_name)
        delay = 20
        next_page = 0
        counter = 0
        product_links = []
        product_descriptions = []
        product_prices = []
        product_discounts = []
        product_sold_count = []
        while next_page == 0:
            page_url = '{}{}{}{}'.format(seller_url, '?page=', counter, '&sortBy=pop')
            driver.get(page_url)
            try:
                WebDriverWait(driver, delay).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.shop-all-product-view')))
                for i in range(5):
                    driver.execute_script("window.scrollTo(0, window.scrollY + 1000)")
                    time.sleep(1.5)
                search_items = driver.find_elements(By.CSS_SELECTOR,'div.shop-search-result-view__item.col-xs-2-4')
                if len(search_items) == 0:
                    next_page = 1
                else:
                    for search_item in search_items:
                        product_link = search_item.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
                        product_description = (search_item.find_element(By.XPATH, '//a/div/div/div[2]/div[1]/div[1]/div').get_attribute(
                            "textContent")).strip()
                        product_price = (
                            search_item.find_element(By.XPATH, '//a/div/div/div[2]/div[2]/div/span[2]').get_attribute(
                                "textContent")).strip()
                        try:
                            product_discount = (search_item.find_element(By.XPATH, 
                                '//a/div/div/div[1]/div/div[2]/div').get_attribute(
                                "textContent")).strip('off')
                        except:
                            product_discount = "0"
                        try:
                            product_sold = (search_item.find_element(By.XPATH, '//a/div/div/div[2]/div[3]/div[2]').get_attribute(
                                "textContent")).strip('Terjual')
                        except:
                            product_sold = "0"
                        product_links.append(product_link)
                        product_descriptions.append(product_description)
                        product_prices.append(product_price)
                        product_discounts.append(product_discount)
                        product_sold_count.append(product_sold)
                counter += 1
                time.sleep(randint(2, 7))
            except TimeoutException:
                next_page = 1
            continue
        data = pd.DataFrame(
            {
                "Name": product_descriptions,
                "Prices": product_prices,
                "Disc": product_discounts,
                "Sold": product_sold_count,
                "Links": product_links,

            })
        

        #data cleaning
        # data['Sold'] = data['Sold'].map(lambda x: float(x.replace('RB','').replace(',','.')) * 1000 if 'RB' in x else x)
        # data.replace(r'^\s*$', np.nan, regex=True, inplace=True)
        # data['Sold'] = data['Sold'].fillna(0).astype('float')

        # data['Disc'] = data['Disc'].str.rstrip('%').astype('float') / 100.0
        # data.sort_values(['Sold','Disc'], ascending=[False, False], inplace=True)
        return data

seller_name = "cerrykebayaubud"

test = Shopee_scraper()
df = test.seller_products(seller_name,)

df

,Name,Prices,Disc,Sold,Links
0,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/kain-metalik-glowsy-jarin...
1,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/CERRY-KEBAYA-KAIN-SEMIPRA...
2,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/kamen-lembaran-bordir-SIT...
3,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/Cerry-Kebaya-kain-FRANCE-...
4,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/CERRY-KEBAYA-SUTRA-CREPE-...
...,...,...,...,...,...
64,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/cerry-kebaya-kamen-bordir...
65,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/cerry-Kebaya-kain-semipra...
66,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/cerry-kebaya-jadi-dark-se...
67,Cerry Kebaya - kain FRANCE MIRROR NEW,134.550,,809,https://shopee.co.id/Cerry-kebaya-Kebaya-jadi-...


In [ ]:
df

,Name,Prices,Disc,Sold,Links
0,,134.550,StarCerry Kebaya UbudAktif 10 menit laluikutic...,mengikuti,https://shopee.co.id/kain-metalik-glowsy-jarin...
1,,134.550,StarCerry Kebaya UbudAktif 10 menit laluikutic...,mengikuti,https://shopee.co.id/CERRY-KEBAYA-KAIN-SEMIPRA...
2,,134.550,StarCerry Kebaya UbudAktif 10 menit laluikutic...,mengikuti,https://shopee.co.id/kamen-lembaran-bordir-SIT...
3,,134.550,StarCerry Kebaya UbudAktif 10 menit laluikutic...,mengikuti,https://shopee.co.id/Cerry-Kebaya-kain-FRANCE-...
4,,134.550,StarCerry Kebaya UbudAktif 10 menit laluikutic...,mengikuti,https://shopee.co.id/CERRY-KEBAYA-SUTRA-CREPE-...
...,...,...,...,...,...
64,,134.550,StarCerry Kebaya UbudAktif 11 menit laluikutic...,mengikuti,https://shopee.co.id/cerry-kebaya-kamen-bordir...
65,,134.550,StarCerry Kebaya UbudAktif 11 menit laluikutic...,mengikuti,https://shopee.co.id/cerry-Kebaya-kain-semipra...
66,,134.550,StarCerry Kebaya UbudAktif 11 menit laluikutic...,mengikuti,https://shopee.co.id/cerry-kebaya-jadi-dark-se...
67,,134.550,StarCerry Kebaya UbudAktif 11 menit laluikutic...,mengikuti,https://shopee.co.id/Cerry-kebaya-Kebaya-jadi-...
